In [1]:
import pandas as pd
import numpy as np
import missingno as msno 
import matplotlib.pyplot as plt
plt.style.use('classic')
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict



In [2]:
df_ml = pd.read_csv('data/df_forML.csv', low_memory=False)
df_ml.shape

(1781974, 174)

In [3]:
x = df_ml.drop(['target'], axis=1)# без целевой переменной
y = df_ml['target']# только с целевой переменной

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)

## Модель Случайного леса

In [15]:
# cлучайный лес
rf_clf = RandomForestClassifier()#инициализируем случайный лес
rf_clf.fit(x_train, y_train) #обучаем модель на тренировочных данных
predicted_train_rf = rf_clf.predict(x_train)#значения показанные моделью на тренировочной выборке
predicted_test_rf = rf_clf.predict(x_test)#значения которые показала модель на тестовой выборке
print(roc_auc_score(y_train, predicted_train_rf))#точность предсказаний модели на тренировочной выборке
print(roc_auc_score(y_test, predicted_test_rf))#точность предсказаний модели на тестовой выборке
confusion_matrix(y_test, predicted_test_rf)#Матрица Ошибок

0.6392817774259136
0.49267562168462375


array([[510630,   8696],
       [ 15235,     32]], dtype=int64)

In [16]:
f_imp_list = list(zip(x_train.columns, rf_clf.feature_importances_))#Ищем нулевые фичи
to_drop = []
for i in f_imp_list:
    if i[1] == 0:
        to_drop.append(i[0])
to_drop #Нулевых фич неоказалось

[]

In [18]:
f_imp_list.sort(key = lambda x: x[1], reverse=True)
f_imp_list[:10]
# 9 из 10 фич оказывающих наибольшее влияние - сгенерированые числовые фичи

[('hour_std', 0.19534788279143478),
 ('device_screen_resolution_std', 0.18795789927134884),
 ('day_std', 0.16912407121589024),
 ('dayofweek_std', 0.09258028740432259),
 ('visit_number_std', 0.08015637844584479),
 ('geo_city_new_std', 0.07259524204095266),
 ('mounth_std', 0.05614188849485666),
 ('day_time_std', 0.039110474524311704),
 ('device_browser_Chrome', 0.00774858985781719),
 ('geo_country_std', 0.005691729997559292)]

### тюнинг модели случайного леса(производился ниже на урезанном датасете, здесь вставлены лучшие параметры)

In [19]:
rf_tuned = RandomForestClassifier(max_features='log2', min_samples_leaf=1, n_estimators=1000)
rf_tuned.fit(x_train, y_train)
predicted_train_rf_tuned = rf_tuned.predict(x_train)#значения показанные моделью на тренировочной выборке
predicted_test_rf_tuned = rf_tuned.predict(x_test)#значения которые показала модель на тестовой выборке
print(f"roc_auc_score для модели случайного леса на тренировочной выборке: "
      f"{roc_auc_score(y_train, predicted_train_rf_tuned):.3f}") #точность предсказаний модели на тренировочной выборке
print(f"roc_auc_score для модели случайного леса на тестовой выборке: "
      f"{roc_auc_score(y_test, predicted_test_rf_tuned):.3f}") #точность предсказаний модели на тестовой выборке

confusion_matrix(y_test, predicted_test_rf_tuned)#Матрица Ошибок

roc_auc_score для модели случайного леса на тренировочной выборке: 0.623
roc_auc_score для модели случайного леса на тестовой выборке: 0.493


array([[510773,   8553],
       [ 15240,     27]], dtype=int64)

## Модель логистической регрессии

In [10]:
logreg = LogisticRegression()#объявляем модель с дефолтными параметрами
logreg.fit(x_train, y_train)#обучаем модель на тренировочной выборке
predicted_train_lr = logreg.predict(x_train)#значения показанные моделью на тренировочной выборке
predicted_test_lr = logreg.predict(x_test)#значения которые показала модель на тестовой выборке
print(roc_auc_score(y_train, predicted_train_lr))#точность предсказаний модели на тренировочной выборке
print(roc_auc_score(y_test, predicted_test_lr))#точность предсказаний модели на тестовой выборке
confusion_matrix(y_test, predicted_test_lr)#Матрица Ошибок

d:\Program files\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5
0.5


array([[519326,      0],
       [ 15267,      0]], dtype=int64)

In [11]:
#Ищем нулевые фичи
coef_logr = list(zip(x_train.columns, logreg.coef_[0]))
to_drop = []
for i in coef_logr:
    if i[1] == 0:
        to_drop.append(i[0])
to_drop

[]

### Тюнинг модели логистической регрессии(производился ниже на урезанном датасете, здесь вставлены лучшие параметры)

In [14]:
logreg_tuned = LogisticRegression(C= 1, penalty= 'l1', solver= 'liblinear')
logreg_tuned.fit(x_train, y_train)
predicted_train_lrt = logreg_tuned.predict(x_train)#значения показанные моделью на тренировочной выборке
predicted_test_lrt = logreg_tuned.predict(x_test)#значения которые показала модель на тестовой выборке
print(roc_auc_score(y_train, predicted_train_lrt))#точность предсказаний модели на тренировочной выборке
print(roc_auc_score(y_test, predicted_test_lrt))#точность предсказаний модели на тестовой выборке
confusion_matrix(y_test, predicted_test_lrt)#Матрица Ошибок

0.5
0.5


array([[519326,      0],
       [ 15267,      0]], dtype=int64)

## Модель многослойного персептрона

In [13]:
mlp = MLPClassifier()
mlp.fit(x_train, y_train)
pred_train_mlp = mlp.predict(x_train)
pred_tets_mlp = mlp.predict(x_test)
print(f"roc_auc_score для модели многослойного персептрона на тренировочной выборке: "
      f"{roc_auc_score(y_train, pred_train_mlp):.3f}")
print(f"roc_auc_score для модели лмногослойного персептрона на тестовой выборке: "
      f"{roc_auc_score(y_test, pred_tets_mlp):.3f}")
confusion_matrix(y_test, pred_tets_mlp )

roc_auc_score для модели многослойного персептрона на тренировочной выборке: 0.500
roc_auc_score для модели лмногослойного персептрона на тестовой выборке: 0.500


array([[519317,      9],
       [ 15260,      7]], dtype=int64)

### Тюнинг модели многослойного персептрона(производился ниже на урезанном датасете, здесь вставлены лучшие параметры)

In [7]:
mlp_tuned = MLPClassifier(activation= 'tanh', alpha= 0.05,  hidden_layer_sizes= (100,), learning_rate= 'adaptive', solver= 'adam')
mlp_tuned.fit(x_train, y_train)
pred_train_mlp = mlp_tuned.predict(x_train)
pred_tets_mlp = mlp_tuned.predict(x_test)
print(f"roc_auc_score для модели многослойного персептрона на тренировочной выборке: "
      f"{roc_auc_score(y_train, pred_train_mlp):.3f}")
print(f"roc_auc_score для модели лмногослойного персептрона на тестовой выборке: "
      f"{roc_auc_score(y_test, pred_tets_mlp):.3f}")
confusion_matrix(y_test, pred_tets_mlp )

roc_auc_score для модели многослойного персептрона на тренировочной выборке: 0.500
roc_auc_score для модели лмногослойного персептрона на тестовой выборке: 0.500


array([[519326,      0],
       [ 15267,      0]], dtype=int64)

## Кросс-валидация

In [6]:
#Модель случайного леса на всей выборке
cv_score_rf_clf = cross_val_score(rf_clf,  x, y, cv=4, scoring='roc_auc')
cv_score_rf_clf.mean(), cv_score_rf_clf.std()

In [8]:
y_pred_rf_clf = cross_val_predict(rf_clf, x, y, cv=4)
conf_mat = confusion_matrix(y, y_pred_rf_clf)
print(conf_mat)

[[1730655    1005]
 [  50216      98]]


In [6]:
#Модель случайного леса на всей выборке(после тюнинга)
cv_score_rf_tuned = cross_val_score(rf_tuned,  x, y, cv=4, scoring='roc_auc')
cv_score_rf_tuned.mean(), cv_score_rf_tuned.std()

(0.6164285565410885, 0.0012651555683117735)

In [9]:
y_pred_rf_tuned = cross_val_predict(rf_tuned, x, y, cv=4)
conf_mat = confusion_matrix(y, y_pred_rf_tuned)
print(conf_mat)

[[1731066     594]
 [  50245      69]]


In [8]:
#Модель многослойного персептрона на всей выборке
cv_score_mlp = cross_val_score(mlp,  x, y, cv=4, scoring='roc_auc')
cv_score_mlp.mean(), cv_score_mlp.std()

(0.6818791454211383, 0.003202318725800435)

In [7]:
y_pred_mlp = cross_val_predict(mlp, x, y, cv=4)
conf_mat = confusion_matrix(y, y_pred_mlp)
print(conf_mat)

[[1731651       9]
 [  50313       1]]


In [23]:
#Модель многослойного персептрона на всей выборке(после тюнинга)
cv_score_mlp_tuned = cross_val_score(mlp_tuned,  x, y, cv=4, scoring='roc_auc')
cv_score_mlp_tuned.mean(), cv_score_mlp_tuned.std()

(0.6711719392750874, 0.00353057532567234)

In [12]:
y_pred_mlp_tuned = cross_val_predict(mlp_tuned, x, y, cv=4)
conf_mat = confusion_matrix(y, y_pred_mlp_tuned)
print(conf_mat)


[[1731660       0]
 [  50314       0]]


In [25]:
#Модель логистической регрессии на всей выборке
cv_score_logreg = cross_val_score(logreg, x, y, cv=4, scoring='roc_auc')
cv_score_logreg.mean(), cv_score_logreg.std()

d:\Program files\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Program files\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optim

(0.6698550862865416, 0.0027133247521936325)

In [26]:
y_pred_logreg = cross_val_predict(logreg, x, y, cv=4)
conf_mat = confusion_matrix(y, y_pred_logreg)
print(conf_mat)

d:\Program files\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Program files\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optim

[[1731660       0]
 [  50314       0]]


In [29]:
#Модель логистической регрессии на всей выборке(после тюнинга)
cv_score_logreg_tuned = cross_val_score(logreg_tuned, x, y, cv=4, scoring='roc_auc')
cv_score_logreg_tuned.mean(), cv_score_logreg_tuned.std()

(0.6704109408025196, 0.002640291314610941)

In [30]:
y_pred_logreg_tuned = cross_val_predict(logreg_tuned, x, y, cv=4)
conf_mat = confusion_matrix(y, y_pred_logreg_tuned)
print(conf_mat)

[[1731660       0]
 [  50314       0]]


## Вывод: 
тюнинг моделей на урезанной выборке результата не дал, лучший показатель roc_auc у MLP с дефолтными параметрами 0.6818, и эта модель предсказывает 1(целевое событитие).

## Тюнинг моделей

In [4]:
#Уменьшаем датасет
df_1 = df_ml[df_ml.target == 1]
df_0 = df_ml[df_ml.target == 0]
df_0 = df_0.iloc[:50314]
df_balanced = pd.concat([df_1, df_0], axis = 0)
df_balanced = df_balanced.sample(frac=1).reset_index(drop = True)

In [5]:
df_balanced.shape

(100628, 175)

In [6]:
x = df_balanced.drop(['target'], axis=1)# без целевой переменной
y = df_balanced['target']# только с целевой переменной

In [7]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [35]:
# cлучайный лес
rf_clf = RandomForestClassifier()#инициализируем случайный лес
rf_clf.fit(x_train, y_train) #обучаем модель на тренировочных данных
predicted_train_rf = rf_clf.predict(x_train)#значения показанные моделью на тренировочной выборке
predicted_test_rf = rf_clf.predict(x_test)#значения которые показала модель на тестовой выборке
print(roc_auc_score(y_train, predicted_train_rf))#точность предсказаний модели на тренировочной выборке
print(roc_auc_score(y_test, predicted_test_rf))#точность предсказаний модели на тестовой выборке
confusion_matrix(y_test, predicted_test_rf)#Матрица Ошибок

0.9798281129917437
0.6303291151380782


array([[9496, 5567],
       [5593, 9533]], dtype=int64)

In [37]:
param_grid = {                  
   'n_estimators': list(range(100, 501, 100)),
   'max_features': ['sqrt', 'log2'],
   'min_samples_leaf': list(range(1, 10)),
   'max_depth':[3,5,10,None] 
            
              } #задаем параметры для тюнинга

In [38]:
rf_clf_gs = RandomForestClassifier()

In [39]:
grid_search_rf = GridSearchCV( 
   estimator=rf_clf_gs,
   param_grid=param_grid,
   scoring='roc_auc',
   verbose=1,
   n_jobs=-1
)

In [40]:
grid_search_rf.fit(x_train, y_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 10, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=1)

In [41]:
best_params = grid_search_rf.best_params
best_params #выводим лучшие параметры для тюнинга

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'n_estimators': 500}

In [46]:
rf_tuned = RandomForestClassifier(max_features='sqrt', min_samples_leaf=3, max_depth= None, n_estimators=500)
rf_tuned.fit(x_train, y_train)
predicted_train_rf_tuned = rf_tuned.predict(x_train)#значения показанные моделью на тренировочной выборке
predicted_test_rf_tuned = rf_tuned.predict(x_test)#значения которые показала модель на тестовой выборке
print(f"roc_auc_score для модели случайного леса на тренировочной выборке: "
      f"{roc_auc_score(y_train, predicted_train_rf_tuned):.3f}") #точность предсказаний модели на тренировочной выборке
print(f"roc_auc_score для модели случайного леса на тестовой выборке: "
      f"{roc_auc_score(y_test, predicted_test_rf_tuned):.3f}") #точность предсказаний модели на тестовой выборке

confusion_matrix(y_test, predicted_test_rf_tuned)#Матрица Ошибок

roc_auc_score для модели случайного леса на тренировочной выборке: 0.740
roc_auc_score для модели случайного леса на тестовой выборке: 0.646


array([[ 9371,  5692],
       [ 4988, 10138]], dtype=int64)

### Тюнинг модели логистической регрессии

In [48]:
logreg = LogisticRegression()#объявляем модель с дефолтными параметрами
logreg.fit(x_train, y_train)#обучаем модель на тренировочной выборке
predicted_train_lr = logreg.predict(x_train)#значения показанные моделью на тренировочной выборке
predicted_test_lr = logreg.predict(x_test)#значения которые показала модель на тестовой выборке
print(roc_auc_score(y_train, predicted_train_lr))#точность предсказаний модели на тренировочной выборке
print(roc_auc_score(y_test, predicted_test_lr))#точность предсказаний модели на тестовой выборке
confusion_matrix(y_test, predicted_test_lr)#Матрица Ошибок

0.6226589036063228
0.6272096043635111


d:\Program files\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[ 8929,  6134],
       [ 5118, 10008]], dtype=int64)

In [51]:
logreg.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [9]:
param_grid = {                  
   'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'penalty':['none', 'elasticnet', 'l1', 'l2'],
              'C':[0.001, 0.01, 0.1, 1, 10, 100]
            
              } #задаем параметры для тюнинга

In [10]:
log_reg_gs = LogisticRegression()

In [11]:
grid_search_log_reg = GridSearchCV( 
   estimator=log_reg_gs,
   param_grid=param_grid,
   scoring='roc_auc',
   verbose=1,
   n_jobs=-1
)

  

In [ ]:
grid_search_log_reg.fit(x_train, y_train)

In [14]:

best_params = grid_search_log_reg.best_params_
best_params #выводим лучшие параметры для тюнинга

{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}

In [15]:
logreg_tuned = LogisticRegression(C= 1, penalty= 'l1', solver= 'liblinear')#объявляем модель с дефолтными параметрами
logreg_tuned.fit(x_train, y_train)#обучаем модель на тренировочной выборке
predicted_train_lr = logreg_tuned.predict(x_train)#значения показанные моделью на тренировочной выборке
predicted_test_lr = logreg_tuned.predict(x_test)#значения которые показала модель на тестовой выборке
print(roc_auc_score(y_train, predicted_train_lr))#точность предсказаний модели на тренировочной выборке
print(roc_auc_score(y_test, predicted_test_lr))#точность предсказаний модели на тестовой выборке
confusion_matrix(y_test, predicted_test_lr)#Матрица Ошибок

0.6240996339017557
0.6243752677254707


array([[8904, 6414],
       [4945, 9926]], dtype=int64)

### Тюнинг модели многослойного персептрона

In [49]:
mlp = MLPClassifier()
mlp.fit(x_train, y_train)
pred_train_mlp = mlp.predict(x_train)
pred_tets_mlp = mlp.predict(x_test)
print(f"roc_auc_score для модели многослойного персептрона на тренировочной выборке: "
      f"{roc_auc_score(y_train, pred_train_mlp):.3f}")
print(f"roc_auc_score для модели лмногослойного персептрона на тестовой выборке: "
      f"{roc_auc_score(y_test, pred_tets_mlp):.3f}")
confusion_matrix(y_test, pred_tets_mlp )

roc_auc_score для модели многослойного персептрона на тренировочной выборке: 0.706
roc_auc_score для модели лмногослойного персептрона на тестовой выборке: 0.618


d:\Program files\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


array([[9325, 5738],
       [5797, 9329]], dtype=int64)

In [16]:
param_grid ={
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [17]:
mlp = MLPClassifier()

In [18]:
grid_search_mlp = GridSearchCV( 
   estimator=mlp,
   param_grid=param_grid,
   scoring='roc_auc',
   verbose=1,
   n_jobs=-1
)

In [ ]:
grid_search_mlp.fit(x_train, y_train)

In [20]:

best_params = grid_search_mlp.best_params_
best_params #выводим лучшие параметры для тюнинга

{'activation': 'tanh',
 'alpha': 0.05,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'adaptive',
 'solver': 'adam'}

In [21]:
mlp_tuned = MLPClassifier(activation= 'tanh', alpha= 0.05,  hidden_layer_sizes= (100,), learning_rate= 'adaptive', solver= 'adam')
mlp_tuned.fit(x_train, y_train)
pred_train_mlp = mlp_tuned.predict(x_train)
pred_tets_mlp = mlp_tuned.predict(x_test)
print(f"roc_auc_score для модели многослойного персептрона на тренировочной выборке: "
      f"{roc_auc_score(y_train, pred_train_mlp):.3f}")
print(f"roc_auc_score для модели лмногослойного персептрона на тестовой выборке: "
      f"{roc_auc_score(y_test, pred_tets_mlp):.3f}")
confusion_matrix(y_test, pred_tets_mlp )

roc_auc_score для модели многослойного персептрона на тренировочной выборке: 0.647
roc_auc_score для модели лмногослойного персептрона на тестовой выборке: 0.632


array([[ 9023,  6295],
       [ 4825, 10046]], dtype=int64)